In [136]:
with open('/home/jeongyeojin/dataset_8224_3.txt') as f:
    data = f.read()
    dna_data = data.split('\n')[:-1]
dna_data

['(+1 +2 -3 -4 -5 -6 +7 -8 -9 -10 +11 +12 +13 +14 +15 +16 +17 -18 -19 +20 -21 -22 +23 +24 +25 +26 +27 -28 +29 -30 -31 -32 +33 +34 +35 -36 +37 +38 +39 +40 -41 +42 -43 +44 -45 -46 +47 -48 +49 +50 -51 -52 -53 +54 +55 -56 +57 +58 -59 +60 +61)',
 '63, 65, 21, 19']

In [1]:
# ex. P : '(-3 +1 +2)
import re

def greedy_sorting(P):
    print_list = lambda x : print("(" + " ".join(["+"+str(s) if s>0 else str(s) for s in x]) + ")")
    approx_reversal_distance = 0
    p_list = [int(s) for s in re.split('[( )]', P)[1:-1]] 
    for i in range(len(p_list)):
        # single flip
        if p_list[i] == -(i+1):
            p_list[i] *= -1
            approx_reversal_distance += 1
            #print_list(p_list)
        elif p_list[i] != (i+1):
            if (i+1) in p_list[i+1:]:
                target_index = p_list.index(i+1)
                p_list = p_list[:i] + [-s for s in p_list[i:target_index+1][::-1]] + p_list[target_index+1:]
                approx_reversal_distance += 1
                #print_list(p_list)
                p_list[i] *= -1
                approx_reversal_distance += 1
                #print_list(p_list)
            elif -(i+1) in p_list[i+1:]:
                target_index = p_list.index(-(i+1))
                p_list = p_list[:i] + [-s for s in p_list[i:target_index+1][::-1]] + p_list[target_index+1:]
                approx_reversal_distance += 1
                #print_list(p_list)
    return approx_reversal_distance
                

In [2]:
# find number of breakpoints
def breakpoint_num(P):
    breakpoints = 0
    p_list = [int(s) for s in re.split('[( )]', P)[1:-1]]  
    p_list = [0] + p_list + [len(p_list)+1]
    for i in range(len(p_list)-1):
        if p_list[i+1] != p_list[i] + 1:
            breakpoints += 1
    return breakpoints
    
    

In [108]:
def chromosome_to_cycle(chromosome):
    nodes = []
    c_list = [int(s) for s in re.split('[( )]', chromosome)[1:-1]] 
    for i in c_list:
        if i > 0:
            nodes.append(2*i -1)
            nodes.append(2*i)
        else:
            nodes.append((-2)*i)
            nodes.append((-2)*i-1)
    nodes_str =  "(" + " ".join([str(s) for s in nodes]) + ")"
    #print(nodes_str)
    return nodes

In [28]:
chromosome_to_cycle('(+4 +5 -6)')

[7, 8, 9, 10, 12, 11]

In [62]:
def cycle_to_chromosome(nodes):
    chromosome = []
    nodes_list = [int(s) for s in re.split('[( )]', nodes)[1:-1]]  
    for i in range(len(nodes_list)//2):
        if nodes_list[2*i] < nodes_list[2*i+1]:
            chromosome.append(nodes_list[2*i+1]//2)
        else:
            chromosome.append((-1)*(nodes_list[2*i]//2))
    chromosome_str = "(" + " ".join(["+"+str(s) if s>0 else str(s) for s in chromosome]) + ")"
    #print(chromosome_str)
    return chromosome_str ,chromosome

In [63]:
cycle_to_chromosome('(1 2 4 3 6 5)')

('(+1 -2 -3)', [1, -2, -3])

In [116]:
def colored_edges(P):
    edges = []
    # list of chromosomes ex.(+1 -2 -3)(+4 +5 -6) -> [(+1 -2 -3), (+4 +5 -6)]
    c_list = [s+")" for s in re.split('[)]',P)[:-1]]
    for x in c_list:
        nodes = chromosome_to_cycle(x) 
        len_n = len(nodes)
        for i in range(1, (len_n//2)+1):
            edges.append((nodes[(2*i-1)%len_n], nodes[(2*i)%len_n]))
    return edges

In [121]:
colored_edges('(+1 -2 -4 +3)')

[(2, 4), (3, 8), (7, 5), (6, 1)]

In [40]:
with open('/home/jeongyeojin/answer.txt', 'w') as f:
    for x in result:
        f.write(str(x)+", ")

In [56]:
colored_edges_to_chromosome([(2, 4), (3, 6), (5, 1)])

[1, 2, 4, 3, 6, 5]

In [64]:
def colored_edges_to_chromosome(edges):
    # input : list that contains colored edges
    nodes_list = []
    for a in edges:
        nodes_list.append(a[0])
        nodes_list.append(a[1])
    nodes_list = nodes_list[len(nodes_list)-1:] + nodes_list[:-1]
    return nodes_list

def graph_to_genome(genome_graph):
    tuple_rx = re.compile("\((\d+),\s*(\d+)\)")
    nodes_list = [] # input (string) 을 list로 변환
    for match in tuple_rx.finditer(genome_graph):
        nodes_list.append((int(match.group(1)), int(match.group(2))))
    cycle_list = [] # 각 cycle을 담을 list
    current_cycle = []
    for x in nodes_list: 
        current_cycle.append(x)
        if x[0] > x[1]: # end of cycle
            cycle_list.append(current_cycle)
            current_cycle = [] # 초기화 for next cycle
    result_str = ''
    for each_cycle in cycle_list:
        input_chromosome = "(" + ' '.join([str(x) for x in colored_edges_to_chromosome(each_cycle)]) + ")"
        # input_chromosome ex. (1 2 4 3 6 5)
        chromosome_str, _ = cycle_to_chromosome(input_chromosome) # ex. (+1 -2 -3)
        result_str += chromosome_str
    return result_str

In [67]:
graph_to_genome(dna_data[0])

'(+1 +2 +3 -4 +5 +6 -7 +8 -9 -10 +11 +12 -13 -14 -15 +16 +17 +18 +19 +20 +21)(-22 -23 +24 +25 +26 -27 -28 -29 +30 +31 -32 -33 -34 -35 +36 +37 -38 +39 -40 +41 +42 -43 +44 +45 +46 +47 +48 -49 +50 +51)(-52 -53 -54 +55 -56 +57 -58 -59 -60 +61 -62 +63 +64 +65 +66 -67 +68 -69 -70 +71 -72 +73 +74 +75)(+76 +77 -78 -79 -80 -81 +82 -83 -84 +85 -86 +87 -88 -89 +90 +91 +92 +93 -94 -95 -96 +97 +98 -99 -100 +101 +102)(+103 -104 -105 +106 -107 +108 +109 -110 -111 +112 +113 +114 -115 +116 -117 +118 -119 +120 -121 +122 -123 +124)(-125 -126 +127 -128 -129 -130 -131 +132 -133 -134 +135 +136 +137 +138 +139 -140 -141 +142 +143 +144 -145 -146 +147 +148 +149 +150)(+151 +152 +153 -154 +155 -156 +157 -158 -159 +160 -161 +162 -163 +164 -165 +166 -167 +168 -169 -170 -171 +172 +173)(-174 +175 +176 -177 +178 +179 -180 -181 -182 +183 -184 +185 -186 -187 -188 +189 -190 +191 +192 -193 +194 +195 -196 +197 +198 -199 -200)'

In [92]:
def two_break_on_genome_graph(genome_graph, i1, i2, i3, i4):
    two_break_points = [i1, i2, i3, i4]
    tuple_rx = re.compile("\((\d+),\s*(\d+)\)")
    nodes_list = [] # input (string) 을 list로 변환
    new_graph = []
    for match in tuple_rx.finditer(genome_graph):
        nodes_list.append((int(match.group(1)), int(match.group(2))))
    for a in nodes_list:
        if a[0] not in two_break_points and a[1] not in two_break_points:
            new_graph.append(a)
        else: # 없어져야 하는 edge
            i_index = two_break_points.index(a[0])
            new_edge = (a[0], two_break_points[(i_index+2)%4])
            new_graph.append(new_edge)
    #new_graph.append((i1, i3))
    #new_graph.append((i2, i4))
    return ", ".join([str(s) for s in new_graph])

In [91]:
two_break_on_genome_graph(dna_data[0], 120, 122, 72, 74)

'(1, 4), (3, 6), (5, 7), (8, 9), (10, 11), (12, 14), (13, 15), (16, 17), (18, 19), (20, 22), (21, 24), (23, 25), (26, 28), (27, 30), (29, 32), (31, 33), (34, 36), (35, 37), (38, 39), (40, 42), (41, 44), (43, 46), (45, 47), (48, 49), (50, 51), (52, 54), (53, 55), (56, 57), (58, 59), (60, 61), (62, 63), (64, 66), (65, 68), (67, 69), (70, 71), (73, 75), (76, 77), (78, 80), (79, 81), (82, 84), (83, 85), (86, 87), (88, 90), (89, 91), (92, 94), (93, 95), (96, 98), (97, 99), (100, 101), (102, 103), (104, 105), (106, 108), (107, 110), (109, 111), (112, 113), (114, 115), (116, 117), (118, 119), (121, 124), (123, 2), (120, 72), (122, 74)'

In [134]:
# ex. (+1 -2 -4 +3), 1, 6, 3, 8 --> (+1 -2)(-3 +4) 
def two_break_on_genome(P, i1, i2, i3, i4):
    # make cycle graph from chromosome 
    a = colored_edges(P) # make colored edges from sorted P 
    # a = [(2, 4), (3, 8), (7, 5), (6, 1)]
    str_colored_edges = ", ".join([str(s) for s in a]) #make into string (함수에 넣기 위해) 
    # (2, 4), (3, 8), (7, 5), (6, 1)
    changed_edges = two_break_on_genome_graph(str_colored_edges, i1, i2, i3, i4) 
    # (2, 4), (3, 1), (7, 5), (6, 8)
    tuple_rx = re.compile("\((\d+),\s*(\d+)\)")
    changed_edges_list = [] # input (string) 을 list로 변환
    for match in tuple_rx.finditer(changed_edges):
        changed_edges_list.append((int(match.group(1)), int(match.group(2))))
        # [(2, 4), (3, 1), (7, 5), (6, 8)]
    #print(changed_edges_list)
    changed_edges_list.sort(key=lambda x: x[0])
    changed_graph = ", ".join([str(s) for s in changed_edges_list])
    return graph_to_genome(changed_graph)

In [135]:
two_break_on_genome('(+1 -2 -4 +3)', 1, 6, 3, 8)

[(2, 4), (3, 1), (7, 5), (6, 8)]


'(+1 -2)(+3 -4)'